# Загрузка данных

In [ ]:
import pandas as pd
import random
# Список offer нужно раскомментировать, если нет исправленного файла offer_example.json
'''offer = [
      "Планшетный компьютер 9.7 Apple iPad 4 A6X 1.4ГГц, 64Гб Flash, Wi-Fi + , GPS, iOS черный (MD512TU/A MD512RS/A)",
      "Планшетный компьютер 10.1 Effire CityNight D10 3G Quad Core MT8389 Cortex A7/1Гб/8Гб/10,1 1280*800/WIFI/Bluetooth/3G/GPS/Android 4.2",
      "Планшетный компьютер 9.7 Apple iPad 32Gb WiFi Space Gray (MP2F2RU/A)",
      "Ноутбук HP Pavilion 15-eh1158ur (601D6EA)",
      "Смартфон Samsung Galaxy Z Fold5 256Gb Phantom Black (SM-F946B)",
      "Смартфон Samsung Galaxy Z Fold5 512Gb Phantom Black (SM-F946B)"
    ]'''

df = pd.read_json('/content/offer_example.json')
model = ['Apple iPad 4 64Gb Wi-Fi', 'Effire CityNight', 'Apple iPad (2017) 32Gb Wi-Fi', 'HP Pavilion', 'Samsung Galaxy Z Fold5 256Gb', 'Samsung Galaxy Z Fold5 512Gb']

Перемешаем наш список, чтобы убедиться в обучении. Так как данные сейчас попарно сходятся offer и model

In [ ]:
random.shuffle(model)
model

['Effire CityNight',
 'Samsung Galaxy Z Fold5 256Gb',
 'HP Pavilion',
 'Apple iPad 4 64Gb Wi-Fi',
 'Samsung Galaxy Z Fold5 512Gb',
 'Apple iPad (2017) 32Gb Wi-Fi']

# Стандартизация данных


In [ ]:
import re

def remove_words(words):
    clear = []
    for word in words:
        # Нижний регистр
        word = word.casefold()
        # Замена гб на gb
        word = word.replace('гб', 'gb')
        # Удаление лишних символов
        word = ''.join(e for e in word if e.isspace() or e.isalnum() )
        # Определение шаблона для поиска ненужных слов
        pattern = r'\b(?!\d+(?:gb))\w+\d+\w+\b'
        # Замена найденных слов на пустую строку и удаление повторяющихся пробелов
        cleaned_words = re.sub(pattern, '', word)
        cleaned_words = re.sub(r'\s+', ' ', cleaned_words)
        clear.append(cleaned_words.strip())

    return clear

Offer - данные, которые мы получаем через json

In [ ]:
# нужно закомментировать offer если нет исправленного файла offer_example.json
offer = df['main']['name']
offer

['Планшетный компьютер 9.7 Apple iPad 4 A6X 1.4ГГц, 64Гб Flash, Wi-Fi + , GPS, iOS черный (MD512TU/A MD512RS/A)',
 'Планшетный компьютер 10.1 Effire CityNight D10 3G Quad Core MT8389 Cortex A7/1Гб/8Гб/10,1 1280*800/WIFI/Bluetooth/3G/GPS/Android 4.2',
 'Планшетный компьютер 9.7 Apple iPad 32Gb WiFi Space Gray (MP2F2RU/A)',
 'Ноутбук HP Pavilion 15-eh1158ur (601D6EA)',
 'Смартфон Samsung Galaxy Z Fold5 256Gb Phantom Black (SM-F946B)',
 'Смартфон Samsung Galaxy Z Fold5 512Gb Phantom Black (SM-F946B)']

offer_clear - стандартизированные данные offer

In [ ]:
offer_clear = remove_words(offer)
offer_clear

['планшетный компьютер 97 apple ipad 4 64gb flash wifi gps ios черный',
 'планшетный компьютер effire citynight 3g quad core cortex 42',
 'планшетный компьютер 97 apple ipad 32gb wifi space gray',
 'ноутбук hp pavilion',
 'смартфон samsung galaxy z fold5 256gb phantom black',
 'смартфон samsung galaxy z fold5 512gb phantom black']

model - данные, которые мы должны матчить с offer

In [ ]:
model

['Effire CityNight',
 'Samsung Galaxy Z Fold5 256Gb',
 'HP Pavilion',
 'Apple iPad 4 64Gb Wi-Fi',
 'Samsung Galaxy Z Fold5 512Gb',
 'Apple iPad (2017) 32Gb Wi-Fi']

model_clear - стандартизированные данные model

In [ ]:
model_clear = remove_words(model)
model_clear

['effire citynight',
 'samsung galaxy z fold5 256gb',
 'hp pavilion',
 'apple ipad 4 64gb wifi',
 'samsung galaxy z fold5 512gb',
 'apple ipad 32gb wifi']

# Обучение kNN

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Векторизация имен и моделей
vectorizer = CountVectorizer()
X_names = vectorizer.fit_transform(offer_clear).toarray()
X_models = vectorizer.transform(model_clear).toarray()

# Создание модели kNN и обучение
knn = NearestNeighbors(n_neighbors=1)
knn.fit(X_models)

# Функция для сопоставления имени с моделью
def match_models(names):
    raw = []
    for name in names:
        name_vector = vectorizer.transform([name]).toarray()
        _, indices = knn.kneighbors(name_vector)
        matched_model = model[indices[0, 0]]
        raw.append(matched_model)
    matching = dict(zip(offer, raw))
    return matching

Результат матчинга решил сохранить в словарь. При необходимости можно будет без проблем добавить в json. Также удобно получить информацию, отдаем offer получаем model. При этом данные сохранены в изначальном виде - без стандартизации. Данные находятся в порядке списка offer

In [ ]:
matching = match_models(offer_clear)
matching

{'Планшетный компьютер 9.7 Apple iPad 4 A6X 1.4ГГц, 64Гб Flash, Wi-Fi + , GPS, iOS черный (MD512TU/A MD512RS/A)': 'Apple iPad 4 64Gb Wi-Fi',
 'Планшетный компьютер 10.1 Effire CityNight D10 3G Quad Core MT8389 Cortex A7/1Гб/8Гб/10,1 1280*800/WIFI/Bluetooth/3G/GPS/Android 4.2': 'Effire CityNight',
 'Планшетный компьютер 9.7 Apple iPad 32Gb WiFi Space Gray (MP2F2RU/A)': 'Apple iPad (2017) 32Gb Wi-Fi',
 'Ноутбук HP Pavilion 15-eh1158ur (601D6EA)': 'HP Pavilion',
 'Смартфон Samsung Galaxy Z Fold5 256Gb Phantom Black (SM-F946B)': 'Samsung Galaxy Z Fold5 256Gb',
 'Смартфон Samsung Galaxy Z Fold5 512Gb Phantom Black (SM-F946B)': 'Samsung Galaxy Z Fold5 512Gb'}

При этом, модель может правильно матчить 2 последних примера, у которых разница лишь в количестве Gb

# Метрика F-мера

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Ожидание
y_true = ['Apple iPad 4 64Gb Wi-Fi', 'Effire CityNight', 'Apple iPad (2017) 32Gb Wi-Fi',  'HP Pavilion', 'Samsung Galaxy Z Fold5 256Gb', 'Samsung Galaxy Z Fold5 512Gb']
# Предсказание
X_pred = list(matching.values())

# Расчет точности, полноты и F-меры
precision = precision_score(y_true, X_pred, average='micro')
recall = recall_score(y_true, X_pred, average='micro')
f1 = f1_score(y_true, X_pred, average='micro')

print("Точность:", precision)
print("Полнота:", recall)
print("F-мера:", f1)

Точность: 1.0
Полнота: 1.0
F-мера: 1.0


На данных примерах, модель работает без ошибок. При дополнении новых данных, есть вероятность того, что модель нужно будет улучшать, для наилучшего результата